In [ ]:
import torch

In [11]:
test = torch.tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 2, 1])
last_eos = (test == 1).nonzero()[-1]
print(last_eos)
print(test[last_eos])
print(test[last_eos+1:])


last_eos == len(test)-1


tensor([14])
tensor([1])
tensor([], dtype=torch.int64)


tensor([True])